# 1. Daten Laden

In [1]:
# Bibliotheken laden
import numpy as np
import pandas as pd

# Dateien von Google Drive bereitstellen
from google.colab import drive
drive.mount('/content/drive')

# Excel auslesen
Datensatz = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Model Engineering/PSP_Jan_Feb_2019.xlsx',header = 0)
Datensatz = Datensatz.rename(columns = {"Unnamed: 0":"id"})

Mounted at /content/drive


# 2. Modellentwicklung & -bewertung

In [2]:
# Bilbiotheken laden
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import roc_auc_score

# Datenaufbereitung
label_encoder = LabelEncoder()

ML_Daten = Datensatz.filter(['amount','success','PSP','country','card','tmsp','3D_secured'], axis=1)

ML_Daten['country'] = label_encoder.fit_transform(ML_Daten['country'])
ML_Daten['PSP'] = label_encoder.fit_transform(ML_Daten['PSP'])
ML_Daten['card'] = label_encoder.fit_transform(ML_Daten['card'])

# Datumswerte in Tag/ Wochentag/ Stunde aufteilen
ML_Daten['weekday'] = ML_Daten['tmsp'].dt.day_name()
ML_Daten['weekday'] = label_encoder.fit_transform(ML_Daten['weekday'])
ML_Daten['day'] = ML_Daten['tmsp'].dt.strftime('%d').astype(int)
ML_Daten['hour'] = ML_Daten['tmsp'].dt.strftime('%H').astype(int)

# Separation in X Merkmale and Zielvariable Y
Y = ML_Daten['success']
X = ML_Daten.filter(['amount','PSP','3D_secured','card','country','weekday','day','hour'], axis=1)

# Aufteilung in Trainings- und Validierungsdatensatz
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

## 2.1 RandForest

In [5]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth = 2)
rfc.fit(x_train, y_train)

y_pred_proba = rfc.predict_proba(x_test)

print('Bewertungsmetriken')
print(f"Vorhersage-Genauigkeit auf Basis der Trainingsdaten: {rfc.score(x_train, y_train)}")
print(f"Vorhersage-Genauigkeit auf Basis der Testdaten: {rfc.score(x_test, y_test)}")
print(f"AUC-Score auf Basis Testdaten: {roc_auc_score(y_test, y_pred_proba[:,0])}")

Bewertungsmetriken
Vorhersage-Genauigkeit auf Basis der Trainingsdaten: 0.7987998413013291
Vorhersage-Genauigkeit auf Basis der Testdaten: 0.7903193810751835
AUC-Score auf Basis Testdaten: 0.3815617246659295


## 2.2 Logistische Regression

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

x_train_scaled = preprocessing.StandardScaler().fit(x_train)

lg = LogisticRegression(max_iter=10)
lg.fit(x_train, y_train)

y_pred_proba = lg.predict_proba(x_test)

print(f"Bewertung der Vorhersage-Genauigkeit auf Basis der Trainingsdaten: {lg.score(x_train, y_train)}")
print(f"Bewertung der Vorhersage-Genauigkeit auf Basis der Testdaten: {lg.score(x_test, y_test)}")
print(f"AUC-Score auf Basis Testdaten: {roc_auc_score(y_test, y_pred_proba[:,0])}")

Bewertung der Vorhersage-Genauigkeit auf Basis der Trainingsdaten: 0.7987998413013291
Bewertung der Vorhersage-Genauigkeit auf Basis der Testdaten: 0.7903193810751835
AUC-Score auf Basis Testdaten: 0.4461363369751475


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 2.3 KNN

In [3]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier()
KNN.fit(x_train, y_train)

y_pred_proba = KNN.predict_proba(x_test)

print(f"Bewertung der Vorhersage-Genauigkeit auf Basis der Trainingsdaten: {KNN.score(x_train, y_train)}")
print(f"Bewertung der Vorhersage-Genauigkeit auf Basis der Testdaten: {KNN.score(x_test, y_test)}")
print(f"AUC-Score auf Basis Testdaten: {roc_auc_score(y_test, y_pred_proba[:,0])}")

Bewertung der Vorhersage-Genauigkeit auf Basis der Trainingsdaten: 0.8170997817893275
Bewertung der Vorhersage-Genauigkeit auf Basis der Testdaten: 0.766712953779012
AUC-Score auf Basis Testdaten: 0.46505606152139306


## 2.4 Support Vektor Machine